In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys
import requests
import urllib
import time
import datetime
import threading
import queue

In [2]:
global days_to_add
days_to_add = 110

In [3]:
def storeInQueue(f):
    def wrapper(*args):
        my_queue.put(f(*args))
    return wrapper

In [4]:
pd.options.mode.chained_assignment = 'raise'

In [5]:
# Use the following site to get list of all publicly traded Stocks
#http://nxcoreapi.com/symbols.php

#nasdaq
#https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download
nqnm = pd.read_csv('nasdaq.csv')
nqnm['Exchange'] = 'Nasdaq'
#nyse
#https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download
nyse = pd.read_csv('nyse.csv')
nyse['Exchange'] = 'NYSE'
#amex
#https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=amex&render=download
amex = pd.read_csv('amex.csv')
amex['Exchange'] = 'AMEX'

In [6]:
amex

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Exchange
0,GOED,1847 Goedeker Inc.,7.3500,$44.92M,2020.0,Consumer Services,Home Furnishings,https://old.nasdaq.com/symbol/goed,AMEX
1,XXII,"22nd Century Group, Inc",0.6164,$85.59M,NaN,Consumer Non-Durables,Farming/Seeds/Milling,https://old.nasdaq.com/symbol/xxii,AMEX
2,FAX,Aberdeen Asia-Pacific Income Fund Inc,4.1900,$1.04B,1986.0,NaN,NaN,https://old.nasdaq.com/symbol/fax,AMEX
3,IAF,Aberdeen Australia Equity Fund Inc,4.8600,$110.53M,NaN,NaN,NaN,https://old.nasdaq.com/symbol/iaf,AMEX
4,AEF,"Aberdeen Emerging Markets Equity Income Fund, ...",6.7800,$344.1M,NaN,NaN,NaN,https://old.nasdaq.com/symbol/aef,AMEX
...,...,...,...,...,...,...,...,...,...
286,WYY,WidePoint Corporation,0.5420,$45.75M,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/wyy,AMEX
287,WTT,"Wireless Telecom Group, Inc.",1.2900,$27.93M,NaN,Capital Goods,Electrical Products,https://old.nasdaq.com/symbol/wtt,AMEX
288,XTNT,"Xtant Medical Holdings, Inc.",0.8671,$11.47M,NaN,Health Care,Biotechnology: Biological Products (No Diagnos...,https://old.nasdaq.com/symbol/xtnt,AMEX
289,ZDGE,"Zedge, Inc.",1.4100,$17.2M,2016.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/zdge,AMEX


In [7]:
stocks = nyse.copy()
stocks = stocks.append(nqnm)
stocks = stocks.append(amex)
# stocks.tail()
# stocks


In [8]:
stocks = stocks[['Symbol', 'Name', 'Exchange', 'Sector', 'industry']]
stocks.reset_index(inplace=True)
stocks = stocks.drop(columns=['index'])
stocks

,Symbol,Name,Exchange,Sector,industry
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments
2,WBAI,500.com Limited,NYSE,Consumer Services,Services-Misc. Amusement & Recreation
3,WUBA,58.com Inc.,NYSE,Technology,"Computer Software: Programming, Data Processing"
4,EGHT,8x8 Inc,NYSE,Technology,EDP Services
...,...,...,...,...,...
7125,WYY,WidePoint Corporation,AMEX,Technology,EDP Services
7126,WTT,"Wireless Telecom Group, Inc.",AMEX,Capital Goods,Electrical Products
7127,XTNT,"Xtant Medical Holdings, Inc.",AMEX,Health Care,Biotechnology: Biological Products (No Diagnos...
7128,ZDGE,"Zedge, Inc.",AMEX,Technology,Computer Software: Prepackaged Software


In [9]:
features = ['dividendRate', 'exDividendDate', 'lastSplitDate', 'lastSplitFactor', 'Buy', 'Sell', 'Hold']
    
for feature in features:
    stocks.loc[:, feature] = np.nan
    
stocks[features] = stocks[features].astype('object')


In [10]:
# # uncomment the following when debugging
# stocks = stocks[0:10]
# stocks

In [11]:
# get quarterly dates for 75 days in future and 75 days in past
def makeQuarterlyDicts():
    
    #get the business days
    quarterly_df = pd.DataFrame()
    start = pd.to_datetime('today') - pd.to_timedelta(days_to_add, unit='d')
    end = pd.to_datetime('today') + pd.to_timedelta(days_to_add, unit='d')
    quarterly_df['Business Dates'] = pd.bdate_range(start=start, end=end)
    
    previous_dict = {}
    future_dict = {}
    holidays = []

    for row in quarterly_df.itertuples():
#         print (row[1])

        date = row[1]
        day = str(row[1])[0:10]
        url = 'https://finance.yahoo.com/calendar/earnings/?day='+day


        # add to previous earnings dates
        try:
            earnings = pd.read_html(url)[0]
            if row[1] < pd.to_datetime('today'):
                for earnings_row in earnings.itertuples():
                    previous_dict[earnings_row[1]] = (date, earnings_row[4], earnings_row[5], earnings_row[6])

            # add to future earnings dates
            else:
                for earnings_row in earnings.itertuples():
                    future_dict[earnings_row[1]] = (date)
        except (ValueError):
            holidays.append(date)
            e = sys.exc_info()[0]
            print(e)
    
    return previous_dict, future_dict

In [12]:
def QuarterlyInfo(stocks):
    initial_time = round(time.time())
    
    previous_dict, future_dict = makeQuarterlyDicts()
    
    previous = pd.DataFrame.from_dict(previous_dict, orient='index',
                                  columns=['Previous_Earnings_Date', 'Expected', 'Actual', 'Surprise'])
    future = pd.DataFrame.from_dict(future_dict, orient = 'index', columns=['Future Earnings Date'])
    
    previous = previous.reset_index()
    future = future.reset_index()
    
    previous.rename(columns={'index': 'Symbol'}, inplace=True)
    future.rename(columns={'index': 'Symbol'}, inplace=True)
    
    stocks = pd.merge(stocks, previous, how='inner')
    stocks = pd.merge(stocks, future, how='left')
    
    print ('Finished quarterly thread in ', (round(time.time())-initial_time)/ 60.0, " minutes!")
    
    return stocks


In [13]:
def getInfo(symbol, info=True, calendar=True, quarterly_financials=True):
    connection_tries=0
    j=0
    
    info = {}
    ticker = yf.Ticker(symbol)           
    info_bool = False
    while 1:
        
        try:
            if info_bool == False:
                info = ticker.info
                recs = ticker.recommendations
                info_bool = True
                           
            return info, recs
            
        except (urllib.error.HTTPError, requests.exceptions.ConnectionError) as e:
            
            if connection_tries < 1:
                print (e, "HTTP Error. Will try again in 10 seconds")
                time.sleep(10)
                connection_tries +=1
            elif connection_tries < 2:
                print (e, "HTTP Error. Will try again in 30 seconds")
                time.sleep(30)
                connection_tries +=1
            elif connection_tries < 4:
                print (e, "HTTP Error. Will try again in 5 minutes")
                time.sleep(300)
                connection_tries +=1
            elif connection_tries < 5:
                print (e, "HTTP Error.  Will try again in 1 hour")
                time.sleep(3600)
                connection_tries +=1
            else:
                connection_tries = 0
                print (e, "HTTP Error.  Giving up.")
                return 'error', 'error'
        except:
            e = sys.exc_info()[0]
#             print('here: ', e)
            if j>2:
                return 'error', 'error'
            j+=1
            

In [14]:
def filterRecs(recs, earnings_date):
    sell = 0
    hold = 0
    buy = 0
    try:
        recs = recs.reset_index()

        for i in range(recs.shape[0]-1, -1, -1):
            time_difference = earnings_date - recs.Date.iloc[i]

            if time_difference < pd.to_timedelta(90, unit='d') and time_difference > pd.to_timedelta(0, unit='d'):
                if recs['To Grade'].iloc[i] in ['Buy', 'Long-Term Buy', 'Overweight', 'OutPerform', 'Strong Buy', 'Outperform', 'Market Outperform', 'Outperformer', 'Positive', 'Sector Outperform']:
                    buy +=1
                elif recs['To Grade'].iloc[i] in ['Peer Perform','Neutral', 'Hold', 'Sector Perform', 'Equal-Weight', 'Market Perform', 'Perform', 'In-Line', 'Sector Weight']:
                    hold +=1
                elif recs['To Grade'].iloc[i] in ['Sell', 'Underperform', 'Underweight', 'Strong Sell', 'Market Underperform', 'Negative', 'Reduce', 'Sector Underperform']:
                    sell +=1
                else:
                    print ('Unknown Recommendation: ', recs['To Grade'].iloc[i])
    except:
        pass
    return buy, hold, sell

In [15]:
# @storeInQueue
def info_thread(stocks, features):
    i = 0
    initial_time = round(time.time())

    bad_symbols = []
    symbols_with_no_return = []

    for row in stocks.itertuples():
#         print ("Info Thread: ", i)    
        if i % 100 == 0:
            print ("It has been: ", (round(time.time())-initial_time)/ 60.0, " minutes in info thread")
        if '/' in row[1] or '.' in row[1] or '/' in row[1]:
            print ('Bad symbol in ', row[1])
            bad_symbols.append(i)       
        else:
            symbol = row[1]
    #         symbol = yf.Ticker(symbol)           

            info, recs = getInfo(symbol)

            if info == 'error':
                symbols_with_no_return.append(row[1])
                print('No return: ', row[1])

            else:

                recent_split = False
                for feature in ['dividendRate', 'exDividendDate', 'lastSplitDate', 'lastSplitFactor']:
                    #try twice in case there's a fail that shouldn't happen

                    try:
                        value = info[feature]
                        
                        if feature == 'lastSplitDate':
                            if datetime.datetime.utcfromtimestamp(value) > pd.to_datetime('today') - pd.to_timedelta(days_to_add, unit='d'):
                                stocks.at[i, feature] = datetime.datetime.utcfromtimestamp(value)
                                recent_split = True
                            else:
                                stocks.at[i, feature] = np.nan
                        elif feature == 'lastSplitFactor':
                            if recent_split:
                                stocks.at[i, feature] = value
                            else:
                                stocks.at[i, feature] = np.nan
                        else:
                            stocks.at[i, feature] = value
                    except:
                        stocks.at[i, feature] = np.nan
                        e = sys.exc_info()[0]
#                         print(e, ": ", feature) 
                
                #populate the buy, sell and hold columns
                #row[10] is previous earnings date
                previous_earnings_date = row[13]
                buy, hold, sell = filterRecs(recs, previous_earnings_date)

                stocks.at[i, 'Buy'] = buy
                stocks.at[i, 'Hold'] = hold
                stocks.at[i, 'Sell'] = sell                        

                    
        i += 1
    print ('Finished info thread in ', (round(time.time())-initial_time)/ 60.0, " minutes!")
    return stocks[features]

In [16]:
stocks = QuarterlyInfo(stocks.copy())

<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
Finished quarterly thread in  3.7333333333333334  minutes!


In [17]:
stocks.shape

(1774, 17)

In [18]:
stocks

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,Sell,Hold,Previous_Earnings_Date,Expected,Actual,Surprise,Future Earnings Date
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-04,-0.1,-0.13,-30,2020-10-28
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27,1.8,1.78,-1.17,2020-10-27
2,WUBA,58.com Inc.,NYSE,Technology,"Computer Software: Programming, Data Processing",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-25,-0.04,2.09,+4744.44,2020-11-16
3,EGHT,8x8 Inc,NYSE,Technology,EDP Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-29,-0.12,-0.07,+43.55,2020-10-28
4,EGHT,8x8 Inc,Nasdaq,Technology,EDP Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-29,-0.12,-0.07,+43.55,2020-10-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,UFAB,"Unique Fabricating, Inc.",AMEX,Capital Goods,Auto Parts:O.E.M.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-24,0.03,-0.12,-500,NaT
1770,UEC,Uranium Energy Corp.,AMEX,Basic Industries,Precious Metals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-08,-,-,-,NaT
1771,VOLT,"Volt Information Sciences, Inc.",AMEX,Technology,Professional Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-09,-0.21,-0.22,-4.76,NaT
1772,WRN,Western Copper and Gold Corporation,AMEX,Basic Industries,Precious Metals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27,-0.01,-0.01,-,NaT


In [19]:
stocks[features] = info_thread(stocks.copy(), features)

It has been:  0.0  minutes in info thread
It has been:  3.683333333333333  minutes in info thread
It has been:  7.35  minutes in info thread
It has been:  11.016666666666667  minutes in info thread
No return:  DSL
No return:  FPF
It has been:  14.95  minutes in info thread
Unknown Recommendation:  Peer Perform
It has been:  18.783333333333335  minutes in info thread
Unknown Recommendation:  Sector Outperform
Unknown Recommendation:  Sector Outperform
Unknown Recommendation:  Peer Perform
Unknown Recommendation:  Peer Perform
It has been:  22.75  minutes in info thread
Unknown Recommendation:  Peer Perform
It has been:  26.616666666666667  minutes in info thread
It has been:  30.35  minutes in info thread
It has been:  34.166666666666664  minutes in info thread
It has been:  37.9  minutes in info thread
It has been:  41.63333333333333  minutes in info thread
It has been:  45.31666666666667  minutes in info thread
No return:  INSUU
HTTP Error 503: Service Unavailable HTTP Error. Will try

In [20]:
stocks.head(10)

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,Sell,Hold,Previous_Earnings_Date,Expected,Actual,Surprise,Future Earnings Date
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,None,None,NaN,NaN,0,0,1,2020-08-04,-0.1,-0.13,-30,2020-10-28
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments,5.88,1597968000,NaN,NaN,1,2,4,2020-07-27,1.8,1.78,-1.17,2020-10-27
2,WUBA,58.com Inc.,NYSE,Technology,"Computer Software: Programming, Data Processing",None,None,NaN,NaN,0,0,0,2020-06-25,-0.04,2.09,+4744.44,2020-11-16
3,EGHT,8x8 Inc,NYSE,Technology,EDP Services,None,None,NaN,NaN,1,1,6,2020-07-29,-0.12,-0.07,+43.55,2020-10-28
4,EGHT,8x8 Inc,Nasdaq,Technology,EDP Services,None,None,NaN,NaN,1,1,6,2020-07-29,-0.12,-0.07,+43.55,2020-10-28
5,AHC,A.H. Belo Corporation,NYSE,Consumer Services,Newspapers/Magazines,0.16,1597276800,NaN,NaN,0,0,0,2020-07-27,-,-,-,NaT
6,AOS,A.O Smith Corporation,NYSE,Consumer Durables,Consumer Electronics/Appliances,0.96,1596067200,NaN,NaN,0,1,5,2020-07-29,0.42,0.45,+7.14,2020-10-27
7,ATEN,"A10 Networks, Inc.",NYSE,Technology,Computer Communications Equipment,None,None,NaN,NaN,0,0,0,2020-07-27,0.08,0.09,+18.42,2020-10-27
8,AIR,AAR Corp.,NYSE,Capital Goods,Aerospace,0.3,1585267200,NaN,NaN,0,0,0,2020-07-20,-0.29,0.26,+189.97,2020-12-17
9,AAN,"Aaron&#39;s, Inc.",NYSE,Technology,Diversified Commercial Services,0.16,1600214400,NaN,NaN,4,0,0,2020-07-28,0.78,1.18,+51.87,2020-11-02


In [21]:
stocks.to_csv('info.csv')

In [22]:
stocks.shape

(1774, 17)

In [25]:
stocks['Previous_Earnings_Date'].unique()

array(['2020-08-04T00:00:00.000000000', '2020-07-27T00:00:00.000000000',
       '2020-06-25T00:00:00.000000000', '2020-07-29T00:00:00.000000000',
       '2020-07-20T00:00:00.000000000', '2020-07-28T00:00:00.000000000',
       '2020-07-21T00:00:00.000000000', '2020-07-15T00:00:00.000000000',
       '2020-07-30T00:00:00.000000000', '2020-08-26T00:00:00.000000000',
       '2020-09-07T00:00:00.000000000', '2020-08-05T00:00:00.000000000',
       '2020-06-24T00:00:00.000000000', '2020-06-29T00:00:00.000000000',
       '2020-07-23T00:00:00.000000000', '2020-08-17T00:00:00.000000000',
       '2020-08-12T00:00:00.000000000', '2020-08-03T00:00:00.000000000',
       '2020-06-11T00:00:00.000000000', '2020-07-22T00:00:00.000000000',
       '2020-07-14T00:00:00.000000000', '2020-08-19T00:00:00.000000000',
       '2020-08-28T00:00:00.000000000', '2020-07-16T00:00:00.000000000',
       '2020-07-13T00:00:00.000000000', '2020-09-08T00:00:00.000000000',
       '2020-08-14T00:00:00.000000000', '2020-08-25